# OpenRefine MC02

This dataset mainly focuses on tides in Puerto Princesa, Palawan from 2017 to 2020. 
The information that will be included in this dataset are:
- date of record
- day of record
- sunrise time
- sunset time
- moonrise time
- moonset time
- tide height
- tide time
- tidal coefficient
- coefficient interpretation
- red tide occurrence in Honda bay and Puerto Princesa Bay

## Import of Packages

In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time as time
import json

import os
from IPython.display import display

## Data Collection
The sites that were scraped are as follows:
- Tide Information:
    - https://tides4fishing.com/as/philippines/puerto-princesa#_tide_table
- Moon Rise and Set
    - http://moon.5deg.com/ph/puerto-princesa/
    
Information on the red tide occurrences were obtained from the red tide bulletin of the Philippine Burueau of Fisheries and Aquatic Resources and was manually encoded into a csv file.

### Tide Information
Information about the tides were obtained from Tides4Fishing. The site's content for historical data could not be accessed through changing the URL since the site was coded in such a way that the data is being accessed through a javascript within the site. Selenium was used to scrape the historical data.

Accessing the Tides4Fishing site and checking of page content.

In [38]:
URL="https://tides4fishing.com/as/philippines/puerto-princesa#_tide_table"

page=requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
soup


<!DOCTYPE html>

<html lang="en">
<!-- mar2 -->
<head>
<title> Tide times and charts for Puerto Princesa, Philippines and weather forecast for fishing in Puerto Princesa in 2021 </title>
<meta charset="utf-8"/>
<meta content="#153152" name="theme-color"/>
<meta content="Tide tables and solunar charts for Puerto Princesa: high tides and low tides, surf reports, sun and moon rising and setting times, lunar phase, fish activity and weather conditions in Puerto Princesa." name="description"/>
<meta content="width=device-width, initial-scale=1, user-scalable=yes" name="viewport"/>
<meta content="Tide times and charts for Puerto Princesa, Philippines and weather forecast for fishing in Puerto Princesa in 2021" property="og:title"/>
<meta content="https://tides4fishing.com/banners/tides4fishing_square_250x250.jpg" property="og:image"/>
<meta content="Tide tables and solunar charts for Puerto Princesa: high tides and low tides, surf reports, sun and moon rising and setting times, lunar phase,

Finding and accessing the table where the desired data was scraped.

In [ ]:
tides4fishing_content=soup.find(id='tabla_mareas_fondo')
tides4fishing_tables=tides4fishing_content.find_all('table')

Trial of scraping of one month and temporarily placing the information obtained into a data frame.

Information can be accessed through the following indices:

|Index|Information|
|-----|-----------|
|  1  |Day of Month (1-31) and Day of Week (Sun, Mon, Tue, Wed, Thu, Fri, Sat)|
|  5  |Sunrise and Sunset Time (24-hour format)|
|  7  |1st Tide (Time and Height in m)|
|  9  |2nd Tide (Time and Height in m)|
| 11  |3rd Tide (Time and Height in m)|
| 13  |4th Tide (Time and Height in m)|
| 15  |Tidal Coefficient|


Test scrape for one month.

In [3]:
info_start=7
t4f_json = []
t4f=tides4fishing_tables[0]

for i in range(info_start, len(t4f.contents),4):
    t4f_info=t4f.contents[i]
    day= " ".join(t4f_info.contents[1].text.strip().split())
    time=" ".join(t4f_info.contents[5].text.strip().split())
    firsttide= " ".join(t4f_info.contents[7].text.strip().split())
    secondtide= " ".join(t4f_info.contents[9].text.strip().split())
    thirdtide= " ".join(t4f_info.contents[11].text.strip().split())
    fourthtide= " ".join(t4f_info.contents[13].text.strip().split())
    coefficient= " ".join(t4f_info.contents[15].text.strip().split())
    t4f_json.append({
        "day": day,
        "time": time,
        "firsttide": firsttide,
        "secondtide": secondtide,
        "thirdtide": thirdtide,
        "fourthtide": fourthtide,
        "coefficient": coefficient
        })
  
##t4f_json
dataframe = pd.DataFrame.from_dict(t4f_json)
dataframe.reset_index(drop=True)
dataframe

#7,11,15.. (30jumps)

,day,time,firsttide,secondtide,thirdtide,fourthtide,coefficient
0,1 Sat,5:48 h 18:16 h,12:13 h 1.6 m,21:12 h -0.3 m,,,63 average
1,2 Sun,5:48 h 18:17 h,13:00 h 1.6 m,22:25 h -0.2 m,,,50 average
2,3 Mon,5:48 h 18:17 h,13:53 h 1.5 m,23:38 h -0.2 m,,,43 low
3,4 Tue,5:47 h 18:17 h,14:56 h 1.4 m,,,,44 low
4,5 Wed,5:47 h 18:17 h,0:41 h -0.1 m,16:13 h 1.3 m,,,51 average
5,6 Thu,5:47 h 18:17 h,1:30 h 0.0 m,17:42 h 1.2 m,,,59 average
6,7 Fri,5:46 h 18:17 h,2:04 h 0.0 m,9:27 h 0.8 m,12:32 h 0.5 m,19:08 h 1.1 m,67 average
7,8 Sat,5:46 h 18:17 h,2:28 h 0.1 m,9:15 h 0.9 m,14:15 h 0.4 m,20:22 h 1.0 m,73 high
8,9 Sun,5:46 h 18:18 h,2:46 h 0.2 m,9:20 h 1.0 m,15:24 h 0.3 m,21:27 h 0.9 m,78 high
9,10 Mon,5:46 h 18:18 h,3:00 h 0.3 m,9:32 h 1.1 m,16:18 h 0.1 m,22:29 h 0.9 m,80 high


Complete scraping of all the data through Selenium and saving each month into a JSON file.

In [9]:
from selenium import webdriver
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time as time
import json


options = Options()
options.add_argument('headless');
options.headless = True
options.add_argument("--window-size=1920,1200")
jsonlist=[]

DRIVER_PATH = r'C:/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://tides4fishing.com/as/philippines/puerto-princesa')
#time.sleep(15)

capabilities = {
 "resolution": "1920x1080"
}
#print(driver.page_source)

driver.find_element_by_xpath("//*[@id='botones_cookies']/a[2]").click()

for x in range(54):
    info_start=6
    t4f_json = []
    filename='t4f'
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    tides4fishing_content=soup.find(id='tabla_mareas_fondo')
    tides4fishing_tables=tides4fishing_content.find_all('table')
    #t4ftable=driver.find_element_by_xpath("//table[@id='tabla_mareas']")
    #t4ftable= soup.find_all('table', attrs={"id":"tabla_mareas"})
    t4f=tides4fishing_tables[0]
    t4f_info= t4f.contents[1]
    
    
    for i in range(info_start, len(t4f_info.contents), 4):
        tidetime1 = "no record"
        tideheight1= "no record"
        tidetime2 = "no record"
        tideheight2= "no record"
        tidetime3 = "no record"
        tideheight3= "no record"
        tidetime4 = "no record"
        tideheight4= "no record"
        
        
        data= t4f_info.contents[i]
        day= " ".join(data.contents[1].text.strip().split())
        time=" ".join(data.contents[5].text.strip().split())
        firsttide= " ".join(data.contents[7].text.strip().split())
        secondtide= " ".join(data.contents[9].text.strip().split())
        thirdtide= " ".join(data.contents[11].text.strip().split())
        fourthtide= " ".join(data.contents[13].text.strip().split())
        coefficient= " ".join(data.contents[15].text.strip().split())
        
        firsttide= firsttide.split()
        if len(firsttide)==4:
            tidetime1= firsttide[0]+" "+firsttide[1]
            tideheight1= firsttide[2]+" "+firsttide[3]
        
        secondtide= secondtide.split()
        if len(secondtide)==4:
            tidetime2= secondtide[0]+" "+secondtide[1]
            tideheight2= secondtide[2]+" "+secondtide[3]
        
        thirdtide= thirdtide.split()
        if len(thirdtide)==4:
            tidetime3= thirdtide[0]+" "+thirdtide[1]
            tideheight3= thirdtide[2]+" "+thirdtide[3]
        
        fourthtide= fourthtide.split()
        if len(fourthtide)==4:
            tidetime4= fourthtide[0]+" "+fourthtide[1]
            tideheight4= fourthtide[2]+" "+fourthtide[3]
        
        t4f_json.append({
            "day": day,
            "time": time,
            "tidetime1": tidetime1,
            "tideheight1": tideheight1,
            "tidetime2": tidetime2,
            "tideheight2": tideheight2,
            "tidetime3": tidetime3,
            "tideheight3": tideheight3,
            "tidetime4":tidetime4,
            "tideheight4":tideheight4,
            "coefficient": coefficient
            })
        
    #print(day)
    #dataframe = pd.DataFrame.from_dict(t4f_json)
    #dataframe.reset_index(drop=True)
    #dataframe
    filenum= str(x)
    formatedjson= json.dumps(t4f_json, indent = 4)
    with open(filename+ '-' + filenum+'.json', "w") as outfile:
        outfile.write(formatedjson)
    
    back= driver.find_element_by_xpath("//*[@id='tabla_mareas_mes']/a[1]")    
    #wait = WebDriverWait(driver, 60);
    #wait.until(EC.invisibilityOfElementLocated(back));
    back.click()
    #time.sleep(10)
    #WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='tabla_mareas_mes']/a[1]"))).click()    
    #back= driver.find_element_by_xpath("//*[@id='tabla_mareas_mes']/a[1]")
    


t4f_json    
#driver.quit()

[{'day': '1 Thu',
  'time': '6:06 h 17:42 h',
  'tidetime1': '6:25 h',
  'tideheight1': '-0.1 m',
  'tidetime2': '13:32 h',
  'tideheight2': '0.7 m',
  'tidetime3': '14:52 h',
  'tideheight3': '0.4 m',
  'tidetime4': '22:50 h',
  'tideheight4': '1.4 m',
  'coefficient': '77 high'},
 {'day': '2 Fri',
  'time': '6:06 h 17:43 h',
  'tidetime1': '7:05 h',
  'tideheight1': '-0.1 m',
  'tidetime2': '23:21 h',
  'tideheight2': '1.4 m',
  'tidetime3': 'no record',
  'tideheight3': 'no record',
  'tidetime4': 'no record',
  'tideheight4': 'no record',
  'coefficient': '73 high'},
 {'day': '3 Sat',
  'time': '6:07 h 17:43 h',
  'tidetime1': '7:47 h',
  'tideheight1': '-0.1 m',
  'tidetime2': '23:54 h',
  'tideheight2': '1.4 m',
  'tidetime3': 'no record',
  'tideheight3': 'no record',
  'tidetime4': 'no record',
  'tideheight4': 'no record',
  'coefficient': '67 average'},
 {'day': '4 Sun',
  'time': '6:07 h 17:43 h',
  'tidetime1': '8:30 h',
  'tideheight1': '-0.1 m',
  'tidetime2': 'no record'

### Moonrise and Moonset

Accessing of Moon.5deg site. All tables with tableline as class within the html was obtained. Checking of index where the desired table of values was also inspected. Desired table is located at index 1.

In [3]:
URL = "http://moon.5deg.com/ph/puerto-princesa/2017/1/"

page=requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table_content=soup.find_all('table', {"class" : "tableline"})

for i in range(len(table_content)):
    print()
    print(f'Index {i} \n {table_content[i]}')


Index 0 
 <table border="0" cellpadding="2" class="tableline">
<tr>
<td align="center"><strong>Mo</strong></td>
<td align="center"><strong>Tu</strong></td>
<td align="center"><strong>We</strong></td>
<td align="center"><strong>Th</strong></td>
<td align="center"><strong>Fr</strong></td>
<td align="center"><strong>Sa</strong></td>
<td align="center"><strong>Su</strong></td>
</tr>
<tr>
<td> </td>
<td> </td>
<td> </td>
<td> </td>
<td> </td>
<td> </td>
<td><a href="../../2017/1/1/" style="margin:2px;" title="2017-1-1">01</a></td>
</tr>
<tr>
<td><a href="../../2017/1/2/" style="margin:2px;" title="2017-1-2">02</a></td>
<td><a href="../../2017/1/3/" style="margin:2px;" title="2017-1-3">03</a></td>
<td><a href="../../2017/1/4/" style="margin:2px;" title="2017-1-4">04</a></td>
<td><a href="../../2017/1/5/" style="margin:2px;" title="2017-1-5">05</a></td>
<td><a href="../../2017/1/6/" style="margin:2px;" title="2017-1-6">06</a></td>
<td><a href="../../2017/1/7/" style="margin:2px;" title="2017

Getting the rows from the table.

In [4]:
moon_table = table_content[1]
moon_rows = moon_table.find_all('tr')
moon_rows

[<tr>
 <td align="center">date</td>
 <td align="center">Day</td>
 <td align="center">Moonrise</td>
 <td align="center">Moonset</td>
 <td align="center">Moon illumination</td>
 <td align="center">Distance</td>
 </tr>,
 <tr>
 <td align="center"><strong>01</strong></td>
 <td align="center">Sunday</td>
 <td align="center"><strong>08:29</strong></td>
 <td align="center"><strong>20:31</strong></td>
 <td align="center"><strong>8%</strong></td>
 <td align="center">387620 km</td>
 </tr>,
 <tr>
 <td align="center"><strong>02</strong></td>
 <td align="center">Monday</td>
 <td align="center"><strong>09:16</strong></td>
 <td align="center"><strong>21:23</strong></td>
 <td align="center"><strong>14.5%</strong></td>
 <td align="center">382892 km</td>
 </tr>,
 <tr>
 <td align="center"><strong>03</strong></td>
 <td align="center">Tuesday</td>
 <td align="center"><strong>10:02</strong></td>
 <td align="center"><strong>22:14</strong></td>
 <td align="center"><strong>22.7%</strong></td>
 <td align="center

Inspecting in what index the information is in.

|Index|Information|
|-----|-----------|
|  1  |Day of Month (1-31)|
|  3  |Day of Week (Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday)|
|  5  |Moonrise Time (24-hour format)|
|  7  |Moonset Time (24-hour format)|
|  11 |Approximate Distance of Moon from Earth in km|

In [5]:
for i in range(len(moon_rows[1].contents)):
    print(f'Index {i} - {moon_rows[1].contents[i]}')

Index 0 - 

Index 1 - <td align="center"><strong>01</strong></td>
Index 2 - 

Index 3 - <td align="center">Sunday</td>
Index 4 - 

Index 5 - <td align="center"><strong>08:29</strong></td>
Index 6 - 

Index 7 - <td align="center"><strong>20:31</strong></td>
Index 8 - 

Index 9 - <td align="center"><strong>8%</strong></td>
Index 10 - 

Index 11 - <td align="center">387620 km</td>
Index 12 - 



Test scrape for one month.

In [7]:
moon = []

yearMonth = URL.split('/')

for r in range(1, len(moon_rows)):
    date = yearMonth[6] + "/" + moon_rows[r].contents[1].text.strip() + "/" + yearMonth[5]
    day = moon_rows[r].contents[3].text.strip()
    moonrise = moon_rows[r].contents[5].text.strip()
    moonset = moon_rows[r].contents[7].text.strip()
    distance = moon_rows[r].contents[11].text.strip().split(" ")[0]
    
    moon.append({
            "date": date,
            "day": day,
            "moonrise": moonrise,
            "moonset": moonset,
            "distance": distance
        })
    
moon

[{'date': '1/01/2017',
  'day': 'Sunday',
  'moonrise': '08:29',
  'moonset': '20:31',
  'distance': '387620'},
 {'date': '1/02/2017',
  'day': 'Monday',
  'moonrise': '09:16',
  'moonset': '21:23',
  'distance': '382892'},
 {'date': '1/03/2017',
  'day': 'Tuesday',
  'moonrise': '10:02',
  'moonset': '22:14',
  'distance': '378186'},
 {'date': '1/04/2017',
  'day': 'Wednesday',
  'moonrise': '10:48',
  'moonset': '23:06',
  'distance': '373755'},
 {'date': '1/05/2017',
  'day': 'Thursday',
  'moonrise': '11:34',
  'moonset': '23:59',
  'distance': '369848'},
 {'date': '1/06/2017',
  'day': 'Friday',
  'moonrise': '12:20',
  'moonset': '00:00',
  'distance': '366699'},
 {'date': '1/07/2017',
  'day': 'Saturday',
  'moonrise': '13:09',
  'moonset': '00:53',
  'distance': '364504'},
 {'date': '1/08/2017',
  'day': 'Sunday',
  'moonrise': '14:01',
  'moonset': '01:50',
  'distance': '363408'},
 {'date': '1/09/2017',
  'day': 'Monday',
  'moonrise': '14:56',
  'moonset': '02:48',
  'distan

Complete scraping code for all months included in dataset and saving into a JSON file.

In [8]:
import requests
import json

from bs4 import BeautifulSoup

URL_List = ["http://moon.5deg.com/ph/puerto-princesa/2017/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/12/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/12/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/12/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/12/"]

moon = []

for url in URL_List:
    page=requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table_content=soup.find_all('table', {"class" : "tableline"})

    moon_table = table_content[1]
    moon_rows = moon_table.find_all('tr')
    moon_rows

    yearMonth = url.split('/')

    for r in range(1, len(moon_rows)):
        date = yearMonth[6] + '/' + moon_rows[r].contents[1].text.strip() + '/' + yearMonth[5]
        day = moon_rows[r].contents[3].text.strip()
        moonrise = moon_rows[r].contents[5].text.strip()
        moonset = moon_rows[r].contents[7].text.strip()
        distance = moon_rows[r].contents[11].text.strip().split(" ")[0]

        moon.append({
                "date": date,
                "day": day,
                "moonrise": moonrise,
                "moonset": moonset,
                "distance": distance
            })

    print("Successful scraping for " + yearMonth[5] + "/" + yearMonth[6])
    
moonjson = json.dumps(moon)
    
#generate json file for all articles
f = open("moon.json", "w")
json_str = str(moonjson)
f.write(json_str)
f.close()

Successful scraping for 2017/1
Successful scraping for 2017/2
Successful scraping for 2017/3
Successful scraping for 2017/4
Successful scraping for 2017/5
Successful scraping for 2017/6
Successful scraping for 2017/7
Successful scraping for 2017/8
Successful scraping for 2017/9
Successful scraping for 2017/10
Successful scraping for 2017/11
Successful scraping for 2017/12
Successful scraping for 2018/1
Successful scraping for 2018/2
Successful scraping for 2018/3
Successful scraping for 2018/4
Successful scraping for 2018/5
Successful scraping for 2018/6
Successful scraping for 2018/7
Successful scraping for 2018/8
Successful scraping for 2018/9
Successful scraping for 2018/10
Successful scraping for 2018/11
Successful scraping for 2018/12
Successful scraping for 2019/1
Successful scraping for 2019/2
Successful scraping for 2019/3
Successful scraping for 2019/4
Successful scraping for 2019/5
Successful scraping for 2019/6
Successful scraping for 2019/7
Successful scraping for 2019/8
Su

## Data Merging

### Opening of JSON and CSV files and placing them into dataframes.

#### Tides4Fishing Data

##### Tide JSON files to dataframes

The scraped data from tides4fishing was placed into separate JSON files per month. The following code tries to open and convert all files into dataframes.

In [20]:
months= ['May', 'April', 'March', 'Feb','Jan','Dec','Nov','Oct','Sep','Aug','Jun','July']
twelvetracker=0
filename='t4f'
year=2021
datalist=[]
dataframelist=[]


for x in range(54):
        
    filenum=str(x)
    with open('v2/t4f jsons/' + filename+ '-' + filenum+'.json', 'r') as openfile:
        t4f_json = json.load(openfile)
    
    datalist.append(t4f_json)
for k in range(54):
    dataframelist.append(pd.DataFrame.from_dict(datalist[k]))

for i in dataframelist:
    if twelvetracker==4:
        year=year-1
        stryear= str(year)
    else:
        stryear=str(year)
    
    print(months[twelvetracker]+ " " + stryear)
    
    display(i)
    print('\n')
    twelvetracker= twelvetracker+1
    if twelvetracker>11:
        twelvetracker=0

#dataframe = pd.DataFrame.from_dict(datalist[1])
#dataframe.reset_index(drop=True)
#dataframe

May 2021


,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient
0,1 Sat,5:48 h 18:16 h,12:13 h,1.6 m,21:12 h,-0.3 m,no record,no record,no record,no record,63 average
1,2 Sun,5:48 h 18:17 h,13:00 h,1.6 m,22:25 h,-0.2 m,no record,no record,no record,no record,50 average
2,3 Mon,5:48 h 18:17 h,13:53 h,1.5 m,23:38 h,-0.2 m,no record,no record,no record,no record,43 low
3,4 Tue,5:47 h 18:17 h,14:56 h,1.4 m,no record,no record,no record,no record,no record,no record,44 low
4,5 Wed,5:47 h 18:17 h,0:41 h,-0.1 m,16:13 h,1.3 m,no record,no record,no record,no record,51 average
5,6 Thu,5:47 h 18:17 h,1:30 h,0.0 m,17:42 h,1.2 m,no record,no record,no record,no record,59 average
6,7 Fri,5:46 h 18:17 h,2:04 h,0.0 m,9:27 h,0.8 m,12:32 h,0.5 m,19:08 h,1.1 m,67 average
7,8 Sat,5:46 h 18:17 h,2:28 h,0.1 m,9:15 h,0.9 m,14:15 h,0.4 m,20:22 h,1.0 m,73 high
8,9 Sun,5:46 h 18:18 h,2:46 h,0.2 m,9:20 h,1.0 m,15:24 h,0.3 m,21:27 h,0.9 m,78 high
9,10 Mon,5:46 h 18:18 h,3:00 h,0.3 m,9:32 h,1.1 m,16:18 h,0.1 m,22:29 h,0.9 m,80 high




April 2021


,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient
0,1 Thu,6:03 h 18:16 h,1:10 h,0.8 m,5:44 h,0.3 m,12:12 h,1.2 m,19:45 h,-0.1 m,84 high
1,2 Fri,6:02 h 18:16 h,2:34 h,0.7 m,5:33 h,0.3 m,12:46 h,1.3 m,21:03 h,-0.2 m,67 average
2,3 Sat,6:02 h 18:16 h,13:27 h,1.4 m,22:30 h,-0.2 m,no record,no record,no record,no record,51 average
3,4 Sun,6:01 h 18:16 h,14:18 h,1.4 m,no record,no record,no record,no record,no record,no record,41 low
4,5 Mon,6:00 h 18:16 h,0:04 h,-0.2 m,15:23 h,1.4 m,no record,no record,no record,no record,42 low
5,6 Tue,6:00 h 18:16 h,1:28 h,-0.2 m,16:44 h,1.3 m,no record,no record,no record,no record,50 average
6,7 Wed,5:59 h 18:15 h,2:29 h,-0.2 m,18:13 h,1.3 m,no record,no record,no record,no record,61 average
7,8 Thu,5:59 h 18:15 h,3:13 h,-0.1 m,19:33 h,1.2 m,no record,no record,no record,no record,72 high
8,9 Fri,5:58 h 18:15 h,3:44 h,-0.1 m,10:32 h,0.7 m,13:24 h,0.3 m,20:38 h,1.2 m,80 high
9,10 Sat,5:58 h 18:15 h,4:07 h,0.0 m,10:24 h,0.8 m,14:49 h,0.3 m,21:33 h,1.1 m,85 high




March 2021


...

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient
0,1 Wed,6:27 h 18:10 h,0:19 h,1.2 m,8:01 h,-0.1 m,14:31 h,0.6 m,18:07 h,0.3 m,80 high
1,2 Thu,6:27 h 18:11 h,1:00 h,1.1 m,8:25 h,0.0 m,14:57 h,0.7 m,19:27 h,0.3 m,71 high
2,3 Fri,6:27 h 18:11 h,1:46 h,1.0 m,8:45 h,0.1 m,15:28 h,0.8 m,21:05 h,0.2 m,62 average
3,4 Sat,6:27 h 18:11 h,2:44 h,0.8 m,9:01 h,0.1 m,16:07 h,0.9 m,23:06 h,0.2 m,56 average
4,5 Sun,6:27 h 18:12 h,4:19 h,0.6 m,9:12 h,0.2 m,16:55 h,1.1 m,no record,no record,55 average
5,6 Mon,6:27 h 18:12 h,1:14 h,0.1 m,7:43 h,0.5 m,8:55 h,0.2 m,17:49 h,1.2 m,60 average
6,7 Tue,6:26 h 18:12 h,2:43 h,-0.1 m,18:47 h,1.3 m,no record,no record,no record,no record,70 high
7,8 Wed,6:26 h 18:12 h,3:41 h,-0.2 m,19:45 h,1.4 m,no record,no record,no record,no record,81 high
8,9 Thu,6:26 h 18:13 h,4:28 h,-0.3 m,20:40 h,1.4 m,no record,no record,no record,no record,90 very high
9,10 Fri,6:26 h 18:13 h,5:08 h,-0.3 m,21:31 h,1.4 m,no record,no record,no record,no record,96 very high




Jan 2016


,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient
0,1 Sun,6:21 h 17:56 h,7:44 h,-0.2 m,23:44 h,1.4 m,no record,no record,no record,no record,78 high
1,2 Mon,6:22 h 17:57 h,8:12 h,-0.2 m,no record,no record,no record,no record,no record,no record,73 high
2,3 Tue,6:22 h 17:57 h,0:20 h,1.3 m,8:47 h,-0.2 m,no record,no record,no record,no record,68 average
3,4 Wed,6:23 h 17:58 h,0:58 h,1.2 m,9:20 h,-0.1 m,no record,no record,no record,no record,62 average
4,5 Thu,6:23 h 17:58 h,1:40 h,1.1 m,9:50 h,0.0 m,17:30 h,0.7 m,20:17 h,0.4 m,58 average
5,6 Fri,6:23 h 17:59 h,2:31 h,1.0 m,10:16 h,0.0 m,17:44 h,0.8 m,22:48 h,0.4 m,58 average
6,7 Sat,6:24 h 17:59 h,3:47 h,0.8 m,10:40 h,0.1 m,18:09 h,1.0 m,no record,no record,62 average
7,8 Sun,6:24 h 18:00 h,1:05 h,0.2 m,6:03 h,0.6 m,11:00 h,0.2 m,18:43 h,1.1 m,70 high
8,9 Mon,6:24 h 18:00 h,2:40 h,0.1 m,8:34 h,0.6 m,11:16 h,0.2 m,19:22 h,1.3 m,79 high
9,10 Tue,6:25 h 18:01 h,3:42 h,-0.1 m,20:05 h,1.4 m,no record,no record,no record,no record,89 high




Dec 2016


,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient
0,1 Thu,6:06 h 17:42 h,6:25 h,-0.1 m,13:32 h,0.7 m,14:52 h,0.4 m,22:50 h,1.4 m,77 high
1,2 Fri,6:06 h 17:43 h,7:05 h,-0.1 m,23:21 h,1.4 m,no record,no record,no record,no record,73 high
2,3 Sat,6:07 h 17:43 h,7:47 h,-0.1 m,23:54 h,1.4 m,no record,no record,no record,no record,67 average
3,4 Sun,6:07 h 17:43 h,8:30 h,-0.1 m,no record,no record,no record,no record,no record,no record,61 average
4,5 Mon,6:08 h 17:43 h,0:28 h,1.3 m,9:14 h,-0.1 m,no record,no record,no record,no record,55 average
5,6 Tue,6:09 h 17:44 h,1:05 h,1.3 m,9:57 h,-0.1 m,no record,no record,no record,no record,51 average
6,7 Wed,6:09 h 17:44 h,1:49 h,1.2 m,10:37 h,0.0 m,no record,no record,no record,no record,52 average
7,8 Thu,6:10 h 17:45 h,2:44 h,1.1 m,11:13 h,0.0 m,19:13 h,0.9 m,22:41 h,0.5 m,58 average
8,9 Fri,6:10 h 17:45 h,4:06 h,0.9 m,11:45 h,0.1 m,19:15 h,0.9 m,no record,no record,68 average
9,10 Sat,6:11 h 17:45 h,0:59 h,0.4 m,6:04 h,0.8 m,12:12 h,0.2 m,19:32 h,1.1 m,80 high


##### Testing of Data Processing
In this part, we will test out separating multiple meaningful information within a column that is meant to be in its own column.

The following are focused on turning the `day` column into `date` (MM/DD/YYYY) that will be used for merging later on.

In [21]:
# December 2020
tempData = dataframelist[5]
tempData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   day          31 non-null     object
 1   time         31 non-null     object
 2   tidetime1    31 non-null     object
 3   tideheight1  31 non-null     object
 4   tidetime2    31 non-null     object
 5   tideheight2  31 non-null     object
 6   tidetime3    31 non-null     object
 7   tideheight3  31 non-null     object
 8   tidetime4    31 non-null     object
 9   tideheight4  31 non-null     object
 10  coefficient  31 non-null     object
dtypes: object(11)
memory usage: 2.8+ KB


In [22]:
tempDays = tempData['day'].str.split()
tempDays

0      [1, Tue]
1      [2, Wed]
2      [3, Thu]
3      [4, Fri]
4      [5, Sat]
5      [6, Sun]
6      [7, Mon]
7      [8, Tue]
8      [9, Wed]
9     [10, Thu]
10    [11, Fri]
11    [12, Sat]
12    [13, Sun]
13    [14, Mon]
14    [15, Tue]
15    [16, Wed]
16    [17, Thu]
17    [18, Fri]
18    [19, Sat]
19    [20, Sun]
20    [21, Mon]
21    [22, Tue]
22    [23, Wed]
23    [24, Thu]
24    [25, Fri]
25    [26, Sat]
26    [27, Sun]
27    [28, Mon]
28    [29, Tue]
29    [30, Wed]
30    [31, Thu]
Name: day, dtype: object

In [24]:
dates = []
dayofWeek =[]

for i in range(len(tempDays)):
    dayofMonth = tempDays[i][0]
    dayofWeek.append(tempDays[i][1])
    date = '12/' + dayofMonth + '/2020'
    dates.append(date)
    
dates
tempData["date"] = dates
# tempData.drop('day', inplace=True, axis=1)
tempData['date'] = pd.to_datetime(tempData['date'])
tempData

,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date
0,6:06 h 17:42 h,5:54 h,-0.1 m,13:15 h,0.7 m,14:09 h,0.4 m,22:17 h,1.4 m,77 high,2020-12-01
1,6:06 h 17:43 h,6:36 h,-0.2 m,22:46 h,1.4 m,no record,no record,no record,no record,74 high,2020-12-02
2,6:07 h 17:43 h,7:20 h,-0.2 m,23:18 h,1.4 m,no record,no record,no record,no record,69 average,2020-12-03
3,6:07 h 17:43 h,8:08 h,-0.2 m,23:53 h,1.4 m,no record,no record,no record,no record,62 average,2020-12-04
4,6:08 h 17:43 h,8:59 h,-0.2 m,no record,no record,no record,no record,no record,no record,56 average,2020-12-05
5,6:09 h 17:44 h,0:31 h,1.4 m,9:50 h,-0.2 m,no record,no record,no record,no record,51 average,2020-12-06
6,6:09 h 17:44 h,1:14 h,1.4 m,10:39 h,-0.1 m,no record,no record,no record,no record,50 average,2020-12-07
7,6:10 h 17:45 h,2:05 h,1.3 m,11:21 h,-0.1 m,no record,no record,no record,no record,55 average,2020-12-08
8,6:10 h 17:45 h,3:08 h,1.1 m,11:55 h,0.0 m,20:03 h,0.8 m,22:44 h,0.5 m,64 average,2020-12-09
9,6:11 h 17:45 h,4:39 h,1.0 m,12:20 h,0.1 m,19:42 h,0.9 m,no record,no record,74 high,2020-12-10


The following are focused on splitting the `time` column into `sunrise` and `sunset` columns.

In [25]:
tempData['time'].str.split()

0     [6:06, h, 17:42, h]
1     [6:06, h, 17:43, h]
2     [6:07, h, 17:43, h]
3     [6:07, h, 17:43, h]
4     [6:08, h, 17:43, h]
5     [6:09, h, 17:44, h]
6     [6:09, h, 17:44, h]
7     [6:10, h, 17:45, h]
8     [6:10, h, 17:45, h]
9     [6:11, h, 17:45, h]
10    [6:11, h, 17:46, h]
11    [6:12, h, 17:46, h]
12    [6:12, h, 17:47, h]
13    [6:13, h, 17:47, h]
14    [6:13, h, 17:47, h]
15    [6:14, h, 17:48, h]
16    [6:14, h, 17:48, h]
17    [6:15, h, 17:49, h]
18    [6:15, h, 17:49, h]
19    [6:16, h, 17:50, h]
20    [6:16, h, 17:50, h]
21    [6:17, h, 17:51, h]
22    [6:17, h, 17:51, h]
23    [6:18, h, 17:52, h]
24    [6:18, h, 17:52, h]
25    [6:19, h, 17:53, h]
26    [6:19, h, 17:53, h]
27    [6:20, h, 17:54, h]
28    [6:20, h, 17:54, h]
29    [6:21, h, 17:55, h]
30    [6:21, h, 17:55, h]
Name: time, dtype: object

In [27]:
sunrise = []
sunset = []

for i in range(len(tempData['time'])):
    sunrise.append(tempData['time'].str.split()[i][0])
    sunset.append(tempData['time'].str.split()[i][2])
    
tempData["sunrise"] = sunrise
tempData["sunset"] = sunset
# tempData.drop('time', inplace=True, axis=1)
tempData

,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset
0,5:54 h,-0.1 m,13:15 h,0.7 m,14:09 h,0.4 m,22:17 h,1.4 m,77 high,2020-12-01,6:06,17:42
1,6:36 h,-0.2 m,22:46 h,1.4 m,no record,no record,no record,no record,74 high,2020-12-02,6:06,17:43
2,7:20 h,-0.2 m,23:18 h,1.4 m,no record,no record,no record,no record,69 average,2020-12-03,6:07,17:43
3,8:08 h,-0.2 m,23:53 h,1.4 m,no record,no record,no record,no record,62 average,2020-12-04,6:07,17:43
4,8:59 h,-0.2 m,no record,no record,no record,no record,no record,no record,56 average,2020-12-05,6:08,17:43
5,0:31 h,1.4 m,9:50 h,-0.2 m,no record,no record,no record,no record,51 average,2020-12-06,6:09,17:44
6,1:14 h,1.4 m,10:39 h,-0.1 m,no record,no record,no record,no record,50 average,2020-12-07,6:09,17:44
7,2:05 h,1.3 m,11:21 h,-0.1 m,no record,no record,no record,no record,55 average,2020-12-08,6:10,17:45
8,3:08 h,1.1 m,11:55 h,0.0 m,20:03 h,0.8 m,22:44 h,0.5 m,64 average,2020-12-09,6:10,17:45
9,4:39 h,1.0 m,12:20 h,0.1 m,19:42 h,0.9 m,no record,no record,74 high,2020-12-10,6:11,17:45


The following are focused on cleaning out the tide columns, removing h and m on entries.

In [12]:
tempData["tidetime1"].str.split()

0     [5:54, h]
1     [6:36, h]
2     [7:20, h]
3     [8:08, h]
4     [8:59, h]
5     [0:31, h]
6     [1:14, h]
7     [2:05, h]
8     [3:08, h]
9     [4:39, h]
10    [1:15, h]
11    [2:47, h]
12    [3:54, h]
13    [4:51, h]
14    [5:45, h]
15    [6:36, h]
16    [7:27, h]
17    [8:17, h]
18    [0:06, h]
19    [0:50, h]
20    [1:35, h]
21    [2:23, h]
22    [3:17, h]
23    [4:36, h]
24    [1:45, h]
25    [3:00, h]
26    [3:51, h]
27    [4:34, h]
28    [5:14, h]
29    [5:54, h]
30    [6:34, h]
Name: tidetime1, dtype: object

In [28]:
for i in range(len(tempData["tidetime1"])):
    if (tempData["tidetime1"][i] != "no record"):
        tempData["tidetime1"][i] = tempData["tidetime1"][i].split()[0]
    if (tempData["tideheight1"][i] != "no record"):
        tempData["tideheight1"][i] = tempData["tideheight1"][i].split()[0]
    if (tempData["tidetime2"][i] != "no record"):
        tempData["tidetime2"][i] = tempData["tidetime2"][i].split()[0]
    if (tempData["tideheight2"][i] != "no record"):
        tempData["tideheight2"][i] = tempData["tideheight2"][i].split()[0]
    if (tempData["tidetime3"][i] != "no record"):
        tempData["tidetime3"][i] = tempData["tidetime3"][i].split()[0]
    if (tempData["tideheight3"][i] != "no record"):
        tempData["tideheight3"][i] = tempData["tideheight3"][i].split()[0]
    if (tempData["tidetime4"][i] != "no record"):
        tempData["tidetime4"][i] = tempData["tidetime4"][i].split()[0]
    if (tempData["tideheight4"][i] != "no record"):
        tempData["tideheight4"][i] = tempData["tideheight4"][i].split()[0]

tempData

<ipython-input-28-f2dc39a9604c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempData["tidetime1"][i] = tempData["tidetime1"][i].split()[0]
<ipython-input-28-f2dc39a9604c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempData["tideheight1"][i] = tempData["tideheight1"][i].split()[0]
<ipython-input-28-f2dc39a9604c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempData["tidetime2"][i] = tempData["tidetime2"][i].split()[0]
<ip

,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset
0,5:54,-0.1,13:15,0.7,14:09,0.4,22:17,1.4,77 high,2020-12-01,6:06,17:42
1,6:36,-0.2,22:46,1.4,no record,no record,no record,no record,74 high,2020-12-02,6:06,17:43
2,7:20,-0.2,23:18,1.4,no record,no record,no record,no record,69 average,2020-12-03,6:07,17:43
3,8:08,-0.2,23:53,1.4,no record,no record,no record,no record,62 average,2020-12-04,6:07,17:43
4,8:59,-0.2,no record,no record,no record,no record,no record,no record,56 average,2020-12-05,6:08,17:43
5,0:31,1.4,9:50,-0.2,no record,no record,no record,no record,51 average,2020-12-06,6:09,17:44
6,1:14,1.4,10:39,-0.1,no record,no record,no record,no record,50 average,2020-12-07,6:09,17:44
7,2:05,1.3,11:21,-0.1,no record,no record,no record,no record,55 average,2020-12-08,6:10,17:45
8,3:08,1.1,11:55,0.0,20:03,0.8,22:44,0.5,64 average,2020-12-09,6:10,17:45
9,4:39,1.0,12:20,0.1,19:42,0.9,no record,no record,74 high,2020-12-10,6:11,17:45


The following are focused on splitting the `coefficient` column information into `coefficient` (holding the coefficient) and `coeff_interpretation` (holding the interpretation of the coefficient) columns.

In [29]:
coefficient = []
coeff_interpretation = []

for i in range(len(tempData['coefficient'])):
    coefficient.append(tempData['coefficient'].str.split()[i][0])
    coeff_interpretation.append(tempData['coefficient'].str.split()[i][1])
    
tempData["coefficient"] = coefficient
tempData["coeff_interpretation"] = coeff_interpretation
# tempData.drop('coefficient', inplace=True, axis=1)
tempData

,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,5:54,-0.1,13:15,0.7,14:09,0.4,22:17,1.4,77,2020-12-01,6:06,17:42,high
1,6:36,-0.2,22:46,1.4,no record,no record,no record,no record,74,2020-12-02,6:06,17:43,high
2,7:20,-0.2,23:18,1.4,no record,no record,no record,no record,69,2020-12-03,6:07,17:43,average
3,8:08,-0.2,23:53,1.4,no record,no record,no record,no record,62,2020-12-04,6:07,17:43,average
4,8:59,-0.2,no record,no record,no record,no record,no record,no record,56,2020-12-05,6:08,17:43,average
5,0:31,1.4,9:50,-0.2,no record,no record,no record,no record,51,2020-12-06,6:09,17:44,average
6,1:14,1.4,10:39,-0.1,no record,no record,no record,no record,50,2020-12-07,6:09,17:44,average
7,2:05,1.3,11:21,-0.1,no record,no record,no record,no record,55,2020-12-08,6:10,17:45,average
8,3:08,1.1,11:55,0.0,20:03,0.8,22:44,0.5,64,2020-12-09,6:10,17:45,average
9,4:39,1.0,12:20,0.1,19:42,0.9,no record,no record,74,2020-12-10,6:11,17:45,high


##### Data Processing Proper
The whole code for accessing the data from JSON files, transforming them into dataframes, and processing the columns and entries is as follows.

In [2]:
months= ['December','November','October','September','August', 'July', 'June','May', 'April', 'March', 'February','January']
months_num = [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
twelvetracker=0
filename='t4f'
year=2021
datalist=[]
dataframelist=[]


for x in range(5, 53):
        
    filenum=str(x)
    with open('v2/t4f jsons/' + filename+ '-' + filenum+'.json', 'r') as openfile:
        t4f_json = json.load(openfile)
    
    datalist.append(t4f_json)
for k in range(48):
    dataframelist.append(pd.DataFrame.from_dict(datalist[k]))

for i in dataframelist:
    if twelvetracker==0:
        year=year-1
        stryear= str(year)
    else:
        stryear=str(year)
    
    print(months[twelvetracker]+ " " + stryear)
    
    #fix days
    dates = []
    dayofWeek =[]
    
    tempDays = i['day'].str.split()

    for j in range(len(tempDays)):
        dayofMonth = tempDays[j][0]
        dayofWeek.append(tempDays[j][1])
        date = str(months_num[twelvetracker]) + '/' + dayofMonth + '/' + stryear
        dates.append(date)

    dates
    i["date"] = dates
    i['date'] = pd.to_datetime(i['date'])
    
    #sun times
    sunrise = []
    sunset = []

    for s in range(len(i['time'])):
        sunrise.append(i['time'].str.split()[s][0])
        sunset.append(i['time'].str.split()[s][2])

    i["sunrise"] = sunrise
    i["sunset"] = sunset
    
    #tides
    for t in range(len(i["tidetime1"])):
        if (i["tidetime1"][t] != "no record"):
            i["tidetime1"][t] = i["tidetime1"][t].split()[0]
        if (i["tideheight1"][t] != "no record"):
            i["tideheight1"][t] = i["tideheight1"][t].split()[0]
        if (i["tidetime2"][t] != "no record"):
            i["tidetime2"][t] = i["tidetime2"][t].split()[0]
        if (i["tideheight2"][t] != "no record"):
            i["tideheight2"][t] = i["tideheight2"][t].split()[0]
        if (i["tidetime3"][t] != "no record"):
            i["tidetime3"][t] = i["tidetime3"][t].split()[0]
        if (i["tideheight3"][t] != "no record"):
            i["tideheight3"][t] = i["tideheight3"][t].split()[0]
        if (i["tidetime4"][t] != "no record"):
            i["tidetime4"][t] = i["tidetime4"][t].split()[0]
        if (i["tideheight4"][t] != "no record"):
            i["tideheight4"][t] = i["tideheight4"][t].split()[0]
    
    #coefficient
    coefficient = []
    coeff_interpretation = []

    for c in range(len(i['coefficient'])):
        coefficient.append(i['coefficient'].str.split()[c][0])
        coeff_interpretation.append(i['coefficient'].str.split()[c][1])

    i["coefficient"] = coefficient
    i["coeff_interpretation"] = coeff_interpretation
    
    display(i)
    print('\n')
    twelvetracker= twelvetracker+1
    if twelvetracker>11:
        twelvetracker=0

December 2020


<ipython-input-2-6e4055103aef>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-2-6e4055103aef>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-2-6e4055103aef>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-2-6e4055103aef>:65: SettingWit

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,1 Tue,6:06 h 17:42 h,5:54,-0.1,13:15,0.7,14:09,0.4,22:17,1.4,77,2020-12-01,6:06,17:42,high
1,2 Wed,6:06 h 17:43 h,6:36,-0.2,22:46,1.4,no record,no record,no record,no record,74,2020-12-02,6:06,17:43,high
2,3 Thu,6:07 h 17:43 h,7:20,-0.2,23:18,1.4,no record,no record,no record,no record,69,2020-12-03,6:07,17:43,average
3,4 Fri,6:07 h 17:43 h,8:08,-0.2,23:53,1.4,no record,no record,no record,no record,62,2020-12-04,6:07,17:43,average
4,5 Sat,6:08 h 17:43 h,8:59,-0.2,no record,no record,no record,no record,no record,no record,56,2020-12-05,6:08,17:43,average
5,6 Sun,6:09 h 17:44 h,0:31,1.4,9:50,-0.2,no record,no record,no record,no record,51,2020-12-06,6:09,17:44,average
6,7 Mon,6:09 h 17:44 h,1:14,1.4,10:39,-0.1,no record,no record,no record,no record,50,2020-12-07,6:09,17:44,average
7,8 Tue,6:10 h 17:45 h,2:05,1.3,11:21,-0.1,no record,no record,no record,no record,55,2020-12-08,6:10,17:45,average
8,9 Wed,6:10 h 17:45 h,3:08,1.1,11:55,0.0,20:03,0.8,22:44,0.5,64,2020-12-09,6:10,17:45,average
9,10 Thu,6:11 h 17:45 h,4:39,1.0,12:20,0.1,19:42,0.9,no record,no record,74,2020-12-10,6:11,17:45,high




November 2020


<ipython-input-2-6e4055103aef>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-2-6e4055103aef>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-2-6e4055103aef>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-2-6e4055103aef>:65: SettingWit

...

<ipython-input-2-6e4055103aef>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-2-6e4055103aef>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-2-6e4055103aef>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-2-6e4055103aef>:65: SettingWit

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,1 Wed,6:27 h 18:10 h,0:19,1.2,8:01,-0.1,14:31,0.6,18:07,0.3,80,2017-02-01,6:27,18:10,high
1,2 Thu,6:27 h 18:11 h,1:00,1.1,8:25,0.0,14:57,0.7,19:27,0.3,71,2017-02-02,6:27,18:11,high
2,3 Fri,6:27 h 18:11 h,1:46,1.0,8:45,0.1,15:28,0.8,21:05,0.2,62,2017-02-03,6:27,18:11,average
3,4 Sat,6:27 h 18:11 h,2:44,0.8,9:01,0.1,16:07,0.9,23:06,0.2,56,2017-02-04,6:27,18:11,average
4,5 Sun,6:27 h 18:12 h,4:19,0.6,9:12,0.2,16:55,1.1,no record,no record,55,2017-02-05,6:27,18:12,average
5,6 Mon,6:27 h 18:12 h,1:14,0.1,7:43,0.5,8:55,0.2,17:49,1.2,60,2017-02-06,6:27,18:12,average
6,7 Tue,6:26 h 18:12 h,2:43,-0.1,18:47,1.3,no record,no record,no record,no record,70,2017-02-07,6:26,18:12,high
7,8 Wed,6:26 h 18:12 h,3:41,-0.2,19:45,1.4,no record,no record,no record,no record,81,2017-02-08,6:26,18:12,high
8,9 Thu,6:26 h 18:13 h,4:28,-0.3,20:40,1.4,no record,no record,no record,no record,90,2017-02-09,6:26,18:13,very
9,10 Fri,6:26 h 18:13 h,5:08,-0.3,21:31,1.4,no record,no record,no record,no record,96,2017-02-10,6:26,18:13,very




January 2017


<ipython-input-2-6e4055103aef>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-2-6e4055103aef>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-2-6e4055103aef>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-2-6e4055103aef>:65: SettingWit

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,1 Sun,6:21 h 17:56 h,7:44,-0.2,23:44,1.4,no record,no record,no record,no record,78,2017-01-01,6:21,17:56,high
1,2 Mon,6:22 h 17:57 h,8:12,-0.2,no record,no record,no record,no record,no record,no record,73,2017-01-02,6:22,17:57,high
2,3 Tue,6:22 h 17:57 h,0:20,1.3,8:47,-0.2,no record,no record,no record,no record,68,2017-01-03,6:22,17:57,average
3,4 Wed,6:23 h 17:58 h,0:58,1.2,9:20,-0.1,no record,no record,no record,no record,62,2017-01-04,6:23,17:58,average
4,5 Thu,6:23 h 17:58 h,1:40,1.1,9:50,0.0,17:30,0.7,20:17,0.4,58,2017-01-05,6:23,17:58,average
5,6 Fri,6:23 h 17:59 h,2:31,1.0,10:16,0.0,17:44,0.8,22:48,0.4,58,2017-01-06,6:23,17:59,average
6,7 Sat,6:24 h 17:59 h,3:47,0.8,10:40,0.1,18:09,1.0,no record,no record,62,2017-01-07,6:24,17:59,average
7,8 Sun,6:24 h 18:00 h,1:05,0.2,6:03,0.6,11:00,0.2,18:43,1.1,70,2017-01-08,6:24,18:00,high
8,9 Mon,6:24 h 18:00 h,2:40,0.1,8:34,0.6,11:16,0.2,19:22,1.3,79,2017-01-09,6:24,18:00,high
9,10 Tue,6:25 h 18:01 h,3:42,-0.1,20:05,1.4,no record,no record,no record,no record,89,2017-01-10,6:25,18:01,high


Dropping the repetitive columns that were already split.

In [3]:
tidesdf = dataframelist[0]
for i in range(1, len(dataframelist)):
    tidesdf = tidesdf.append(dataframelist[i])
    
tidesdf.drop('day', inplace=True, axis=1)
tidesdf.drop('time', inplace=True, axis=1)
    
tidesdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 30
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tidetime1             1461 non-null   object        
 1   tideheight1           1461 non-null   object        
 2   tidetime2             1461 non-null   object        
 3   tideheight2           1461 non-null   object        
 4   tidetime3             1461 non-null   object        
 5   tideheight3           1461 non-null   object        
 6   tidetime4             1461 non-null   object        
 7   tideheight4           1461 non-null   object        
 8   coefficient           1461 non-null   object        
 9   date                  1461 non-null   datetime64[ns]
 10  sunrise               1461 non-null   object        
 11  sunset                1461 non-null   object        
 12  coeff_interpretation  1461 non-null   object        
dtypes: datetime64[ns](1)

In [41]:
# for i in range(len(tempData['tideheight1'])):
#     tides = []
# #   -1 = no record, 0 = low, 1 = high
#     tides_interpret = [-1,-1,-1,-1]
#     if (tempData['tideheight1'][i] != "no record"):
#         tides.append(pd.to_numeric(tempData['tideheight1'][i]))
#     if (tempData['tideheight2'][i] != "no record"):
#         tides.append(pd.to_numeric(tempData['tideheight2'][i]))
#     if (tempData['tideheight3'][i] != "no record"):
#         tides.append(pd.to_numeric(tempData['tideheight3'][i]))
#     if (tempData['tideheight4'][i] != "no record"):
#         tides.append(pd.to_numeric(tempData['tideheight4'][i]))
        
#     if (len(tides) == 1):
#         tides_interpret[0] = 1
#     elif (len(tides) > 1):
#         if (len(tides) == 2):
#             tides_interpret[tides.index(max(tides))] = 1
#             tides_interpret[tides.index(min(tides))] = 0
#         elif (len(tides) == 3):
#             if (tides[0] > tides[1]):
#                 tides_interpret[0] = 1
#                 tides_interpret[1] = 0
#                 tides_interpret[2] = 1
#             elif (tides[0] < tides[1]):
#                 tides_interpret[0] = 0
#                 tides_interpret[1] = 1
#                 tides_interpret[2] = 0
#         elif (len(tides) == 4):
#             tides_interpret[tides.index(max(tides))] = 1
#             tides.remove(max(tides))
#             tides_interpret[tides.index(max(tides))] = 1
#             tides.remove(max(tides))
#             tides_interpret[tides.index(max(tides))] = 0
#             tides.remove(max(tides))
#             tides_interpret[tides.index(max(tides))] = 0
            
# # TIDE 1
#     if (tides_interpret[0] == 1):
#         tempData["tide1_interpretation"][i] = "high"
#     elif (tides_interpret[0] == 0):
#         tempData["tide1_interpretation"][i] = "low"
#     elif (tides_interpret[0] == -1):
#         tempData["tide1_interpretation"][i] = "no record"
# # TIDE 2       
#     if (tides_interpret[0] == 1):
#         tempData["tide2_interpretation"][i] = "high"
#     elif (tides_interpret[0] == 0):
#         tempData["tide2_interpretation"][i] = "low"
#     elif (tides_interpret[0] == -1):
#         tempData["tide2_interpretation"][i] = "no record"
# # TIDE 3     
#     if (tides_interpret[0] == 1):
#         tempData["tide3_interpretation"][i] = "high"
#     elif (tides_interpret[0] == 0):
#         tempData["tide3_interpretation"][i] = "low"
#     elif (tides_interpret[0] == -1):
#         tempData["tide3_interpretation"][i] = "no record"
# # TIDE 4      
#     if (tides_interpret[0] == 1):
#         tempData["tide4_interpretation"][i] = "high"
#     elif (tides_interpret[0] == 0):
#         tempData["tide4_interpretation"][i] = "low"
#     elif (tides_interpret[0] == -1):
#         tempData["tide4_interpretation"][i] = "no record"
            

KeyError: 'tide1_interpretation'

#### Moon Data

##### Moon JSON file to dataframe

In [4]:
moonframe = pd.read_json('moon.json')
moonframe

,date,day,moonrise,moonset,distance
0,2017-01-01,Sunday,08:29,20:31,387620
1,2017-01-02,Monday,09:16,21:23,382892
2,2017-01-03,Tuesday,10:02,22:14,378186
3,2017-01-04,Wednesday,10:48,23:06,373755
4,2017-01-05,Thursday,11:34,23:59,369848
...,...,...,...,...,...
1456,2020-12-27,Sunday,15:36,03:39,403274
1457,2020-12-28,Monday,16:22,04:29,400801
1458,2020-12-29,Tuesday,17:12,05:21,397535
1459,2020-12-30,Wednesday,18:05,06:14,393608


In [69]:
moonframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1461 non-null   datetime64[ns]
 1   day       1461 non-null   object        
 2   moonrise  1461 non-null   object        
 3   moonset   1461 non-null   object        
 4   distance  1461 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 57.2+ KB


#### Red Tide Data

##### Red Tide CSV file to dataframe

In [5]:
redtideFrame = pd.read_csv('RedTide.csv')
redtideFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           1462 non-null   object
 1   redtide_pp     1462 non-null   int64 
 2   redtide_honda  1462 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 34.4+ KB


`date` column not yet in datetime format.
Transforming `date` column from object to datetime format.

In [6]:
redtideFrame['date'] = pd.to_datetime(redtideFrame['date'])
redtideFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1462 non-null   datetime64[ns]
 1   redtide_pp     1462 non-null   int64         
 2   redtide_honda  1462 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 34.4 KB


In [72]:
redtideFrame

,date,redtide_pp,redtide_honda
0,2016-12-31,0,0
1,2017-01-01,0,0
2,2017-01-02,0,0
3,2017-01-03,0,0
4,2017-01-04,0,0
...,...,...,...
1457,2020-12-27,1,1
1458,2020-12-28,1,1
1459,2020-12-29,1,1
1460,2020-12-30,1,1


Merging of Moon dataframe with Red Tide dataframe through left join using `date` of Moon as key.

In [72]:
# df1.merge(df2, how='left', on='a')

finalDF = moonframe.merge(redtideFrame, how='left', on='date')
finalDF

,date,day,moonrise,moonset,distance,redtide_pp,redtide_honda
0,2017-01-01,Sunday,08:29,20:31,387620,0,0
1,2017-01-02,Monday,09:16,21:23,382892,0,0
2,2017-01-03,Tuesday,10:02,22:14,378186,0,0
3,2017-01-04,Wednesday,10:48,23:06,373755,0,0
4,2017-01-05,Thursday,11:34,23:59,369848,0,0
...,...,...,...,...,...,...,...
1456,2020-12-27,Sunday,15:36,03:39,403274,1,1
1457,2020-12-28,Monday,16:22,04:29,400801,1,1
1458,2020-12-29,Tuesday,17:12,05:21,397535,1,1
1459,2020-12-30,Wednesday,18:05,06:14,393608,1,1


Merging of final dataframe with tide dataframe through left join using `date` of final dataframe as key.

In [73]:
finalDF = finalDF.merge(tidesdf, how='left', on='date')
finalDF

,date,day,moonrise,moonset,distance,redtide_pp,redtide_honda,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,sunrise,sunset,coeff_interpretation
0,2017-01-01,Sunday,08:29,20:31,387620,0,0,7:44,-0.2,23:44,1.4,no record,no record,no record,no record,78,6:21,17:56,high
1,2017-01-02,Monday,09:16,21:23,382892,0,0,8:12,-0.2,no record,no record,no record,no record,no record,no record,73,6:22,17:57,high
2,2017-01-03,Tuesday,10:02,22:14,378186,0,0,0:20,1.3,8:47,-0.2,no record,no record,no record,no record,68,6:22,17:57,average
3,2017-01-04,Wednesday,10:48,23:06,373755,0,0,0:58,1.2,9:20,-0.1,no record,no record,no record,no record,62,6:23,17:58,average
4,2017-01-05,Thursday,11:34,23:59,369848,0,0,1:40,1.1,9:50,0.0,17:30,0.7,20:17,0.4,58,6:23,17:58,average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2020-12-27,Sunday,15:36,03:39,403274,1,1,3:51,0.0,10:10,0.6,12:25,0.3,20:20,1.2,67,6:19,17:53,average
1457,2020-12-28,Monday,16:22,04:29,400801,1,1,4:34,-0.1,20:51,1.3,no record,no record,no record,no record,72,6:20,17:54,high
1458,2020-12-29,Tuesday,17:12,05:21,397535,1,1,5:14,-0.2,21:24,1.4,no record,no record,no record,no record,76,6:20,17:54,high
1459,2020-12-30,Wednesday,18:05,06:14,393608,1,1,5:54,-0.2,21:59,1.4,no record,no record,no record,no record,78,6:21,17:55,high


Rearranging the columns of the dataframe.

In [74]:
finalDF = finalDF[['date', 'day', 'sunrise', 'sunset', 'moonrise', 'moonset', 'distance','tidetime1','tideheight1','tidetime2','tideheight2','tidetime3','tideheight3','tidetime4','tideheight4','coefficient','coeff_interpretation','redtide_pp','redtide_honda']]
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   object        
 3   sunset                1461 non-null   object        
 4   moonrise              1461 non-null   object        
 5   moonset               1461 non-null   object        
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   object        
 8   tideheight1           1461 non-null   object        
 9   tidetime2             1461 non-null   object        
 10  tideheight2           1461 non-null   object        
 11  tidetime3             1461 non-null   object        
 12  tideheight3           1461 non-null   object        
 13  tidetime4         

In [75]:
finalDF['coefficient'] = finalDF['coefficient'].astype(str).astype(int)
finalDF['tideheight1'] = finalDF['tideheight1'].astype(str).astype(float)
# finalDF['tideheight1'] = finalDF['tideheight2'].astype(str).astype(float)
# finalDF['tideheight1'] = finalDF['tideheight3'].astype(str).astype(float)
# finalDF['tideheight1'] = finalDF['tideheight4'].astype(str).astype(float)
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   object        
 3   sunset                1461 non-null   object        
 4   moonrise              1461 non-null   object        
 5   moonset               1461 non-null   object        
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   object        
 8   tideheight1           1461 non-null   float64       
 9   tidetime2             1461 non-null   object        
 10  tideheight2           1461 non-null   object        
 11  tidetime3             1461 non-null   object        
 12  tideheight3           1461 non-null   object        
 13  tidetime4         

In [76]:
for i in range(len(finalDF['sunrise'])):
    # sunrise time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['sunrise'][i]
    time = pd.to_datetime(time)
    finalDF['sunrise'][i] = time
    # sunset time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['sunset'][i]
    time = pd.to_datetime(time)
    finalDF['sunset'][i] = time
    # moonrise time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['moonrise'][i]
    time = pd.to_datetime(time)
    finalDF['moonrise'][i] = time
    # moonset time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['moonset'][i]
    time = pd.to_datetime(time)
    finalDF['moonset'][i] = time
    
finalDF

<ipython-input-76-37dab05b371a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['sunrise'][i] = time
<ipython-input-76-37dab05b371a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['sunset'][i] = time
<ipython-input-76-37dab05b371a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['moonrise'][i] = time
<ipython-input-76-37dab05b371a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,date,day,sunrise,sunset,moonrise,moonset,distance,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,coeff_interpretation,redtide_pp,redtide_honda
0,2017-01-01,Sunday,2017-01-01 06:21:00,2017-01-01 17:56:00,2017-01-01 08:29:00,2017-01-01 20:31:00,387620,7:44,-0.2,23:44,1.4,no record,no record,no record,no record,78,high,0,0
1,2017-01-02,Monday,2017-01-02 06:22:00,2017-01-02 17:57:00,2017-01-02 09:16:00,2017-01-02 21:23:00,382892,8:12,-0.2,no record,no record,no record,no record,no record,no record,73,high,0,0
2,2017-01-03,Tuesday,2017-01-03 06:22:00,2017-01-03 17:57:00,2017-01-03 10:02:00,2017-01-03 22:14:00,378186,0:20,1.3,8:47,-0.2,no record,no record,no record,no record,68,average,0,0
3,2017-01-04,Wednesday,2017-01-04 06:23:00,2017-01-04 17:58:00,2017-01-04 10:48:00,2017-01-04 23:06:00,373755,0:58,1.2,9:20,-0.1,no record,no record,no record,no record,62,average,0,0
4,2017-01-05,Thursday,2017-01-05 06:23:00,2017-01-05 17:58:00,2017-01-05 11:34:00,2017-01-05 23:59:00,369848,1:40,1.1,9:50,0.0,17:30,0.7,20:17,0.4,58,average,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2020-12-27,Sunday,2020-12-27 06:19:00,2020-12-27 17:53:00,2020-12-27 15:36:00,2020-12-27 03:39:00,403274,3:51,0.0,10:10,0.6,12:25,0.3,20:20,1.2,67,average,1,1
1457,2020-12-28,Monday,2020-12-28 06:20:00,2020-12-28 17:54:00,2020-12-28 16:22:00,2020-12-28 04:29:00,400801,4:34,-0.1,20:51,1.3,no record,no record,no record,no record,72,high,1,1
1458,2020-12-29,Tuesday,2020-12-29 06:20:00,2020-12-29 17:54:00,2020-12-29 17:12:00,2020-12-29 05:21:00,397535,5:14,-0.2,21:24,1.4,no record,no record,no record,no record,76,high,1,1
1459,2020-12-30,Wednesday,2020-12-30 06:21:00,2020-12-30 17:55:00,2020-12-30 18:05:00,2020-12-30 06:14:00,393608,5:54,-0.2,21:59,1.4,no record,no record,no record,no record,78,high,1,1


In [77]:
finalDF['sunrise'] =pd.to_datetime(finalDF['sunrise'])
finalDF['sunset'] =pd.to_datetime(finalDF['sunset'])
finalDF['moonrise'] =pd.to_datetime(finalDF['moonrise'])
finalDF['moonset'] =pd.to_datetime(finalDF['moonset'])
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   datetime64[ns]
 3   sunset                1461 non-null   datetime64[ns]
 4   moonrise              1461 non-null   datetime64[ns]
 5   moonset               1461 non-null   datetime64[ns]
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   object        
 8   tideheight1           1461 non-null   float64       
 9   tidetime2             1461 non-null   object        
 10  tideheight2           1461 non-null   object        
 11  tidetime3             1461 non-null   object        
 12  tideheight3           1461 non-null   object        
 13  tidetime4         

In [64]:
def removeNoRecords(value):
    result = re.sub("no record", "99999", value)
    result = result.strip()
    return result

In [65]:
finalDF[['tideheight2', 'tideheight3', 'tideheight4']] = finalDF[['tideheight2', 'tideheight3', 'tideheight4']].applymap(removeNoRecords)

In [66]:
finalDF['tideheight2'] = finalDF['tideheight2'].astype(float)
finalDF['tideheight3'] = finalDF['tideheight3'].astype(float)
finalDF['tideheight4'] = finalDF['tideheight4'].astype(float)

In [78]:
finalDF['tideheight2'] = pd.to_numeric(finalDF['tideheight2'], errors='coerce')
finalDF['tideheight3'] = pd.to_numeric(finalDF['tideheight3'], errors='coerce')
finalDF['tideheight4'] = pd.to_numeric(finalDF['tideheight4'], errors='coerce')

In [79]:
for i in range(len(finalDF['tidetime1'])):
    # tide 1 time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime1'][i]
    time = pd.to_datetime(time)
    finalDF['tidetime1'][i] = time
    # tide 2 time conversion to datetime data type
    if (finalDF['tidetime2'][i] != "no record"):
        time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime2'][i]
        time = pd.to_datetime(time)
        finalDF['tidetime2'][i] = time
    # tide 3 time conversion to datetime data type
    if (finalDF['tidetime3'][i] != "no record"):
        time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime3'][i]
        time = pd.to_datetime(time)
        finalDF['tidetime3'][i] = time
    # tide 4 time conversion to datetime data type
    if (finalDF['tidetime4'][i] != "no record"):
        time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime4'][i]
        time = pd.to_datetime(time)
        finalDF['tidetime4'][i] = time

<ipython-input-79-30218544e7bc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetime1'][i] = time
<ipython-input-79-30218544e7bc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetime2'][i] = time
<ipython-input-79-30218544e7bc>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetime3'][i] = time
<ipython-input-79-30218544e7bc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [80]:
finalDF['tidetime1'] = pd.to_datetime(finalDF['tidetime1'], errors='coerce')
finalDF['tidetime2'] = pd.to_datetime(finalDF['tidetime2'], errors='coerce')
finalDF['tidetime3'] = pd.to_datetime(finalDF['tidetime3'], errors='coerce')
finalDF['tidetime4'] = pd.to_datetime(finalDF['tidetime4'], errors='coerce')

In [81]:
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   datetime64[ns]
 3   sunset                1461 non-null   datetime64[ns]
 4   moonrise              1461 non-null   datetime64[ns]
 5   moonset               1461 non-null   datetime64[ns]
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   datetime64[ns]
 8   tideheight1           1461 non-null   float64       
 9   tidetime2             1411 non-null   datetime64[ns]
 10  tideheight2           1411 non-null   float64       
 11  tidetime3             811 non-null    datetime64[ns]
 12  tideheight3           811 non-null    float64       
 13  tidetime4         

### Final Dataset

In [82]:
finalDF

,date,day,sunrise,sunset,moonrise,moonset,distance,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,coeff_interpretation,redtide_pp,redtide_honda
0,2017-01-01,Sunday,2017-01-01 06:21:00,2017-01-01 17:56:00,2017-01-01 08:29:00,2017-01-01 20:31:00,387620,2017-01-01 07:44:00,-0.2,2017-01-01 23:44:00,1.4,NaT,NaN,NaT,NaN,78,high,0,0
1,2017-01-02,Monday,2017-01-02 06:22:00,2017-01-02 17:57:00,2017-01-02 09:16:00,2017-01-02 21:23:00,382892,2017-01-02 08:12:00,-0.2,NaT,NaN,NaT,NaN,NaT,NaN,73,high,0,0
2,2017-01-03,Tuesday,2017-01-03 06:22:00,2017-01-03 17:57:00,2017-01-03 10:02:00,2017-01-03 22:14:00,378186,2017-01-03 00:20:00,1.3,2017-01-03 08:47:00,-0.2,NaT,NaN,NaT,NaN,68,average,0,0
3,2017-01-04,Wednesday,2017-01-04 06:23:00,2017-01-04 17:58:00,2017-01-04 10:48:00,2017-01-04 23:06:00,373755,2017-01-04 00:58:00,1.2,2017-01-04 09:20:00,-0.1,NaT,NaN,NaT,NaN,62,average,0,0
4,2017-01-05,Thursday,2017-01-05 06:23:00,2017-01-05 17:58:00,2017-01-05 11:34:00,2017-01-05 23:59:00,369848,2017-01-05 01:40:00,1.1,2017-01-05 09:50:00,0.0,2017-01-05 17:30:00,0.7,2017-01-05 20:17:00,0.4,58,average,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2020-12-27,Sunday,2020-12-27 06:19:00,2020-12-27 17:53:00,2020-12-27 15:36:00,2020-12-27 03:39:00,403274,2020-12-27 03:51:00,0.0,2020-12-27 10:10:00,0.6,2020-12-27 12:25:00,0.3,2020-12-27 20:20:00,1.2,67,average,1,1
1457,2020-12-28,Monday,2020-12-28 06:20:00,2020-12-28 17:54:00,2020-12-28 16:22:00,2020-12-28 04:29:00,400801,2020-12-28 04:34:00,-0.1,2020-12-28 20:51:00,1.3,NaT,NaN,NaT,NaN,72,high,1,1
1458,2020-12-29,Tuesday,2020-12-29 06:20:00,2020-12-29 17:54:00,2020-12-29 17:12:00,2020-12-29 05:21:00,397535,2020-12-29 05:14:00,-0.2,2020-12-29 21:24:00,1.4,NaT,NaN,NaT,NaN,76,high,1,1
1459,2020-12-30,Wednesday,2020-12-30 06:21:00,2020-12-30 17:55:00,2020-12-30 18:05:00,2020-12-30 06:14:00,393608,2020-12-30 05:54:00,-0.2,2020-12-30 21:59:00,1.4,NaT,NaN,NaT,NaN,78,high,1,1


## EDA